# 04_01: Creating a GraphRAG pipeline with LangChain to query your data

In [ ]:
import os
from dotenv import load_dotenv

from langchain.prompts import PromptTemplate
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_openai import ChatOpenAI

In [ ]:
load_dotenv(dotenv_path='../.env')

URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_USER")
PWD = os.getenv("NEO4J_PWD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model_name="gpt-4o")
llm_transformer = LLMGraphTransformer(llm=llm)
graph = Neo4jGraph(url=URI, username=USER, password=PWD)

In [ ]:
graph.query('MATCH (n) RETURN COUNT(n) as TOTAL')

In [ ]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to
answer questions about skiing.
Convert the user's question based on the schema.
When you are presented with query properties such as id's like "grass skiing",
be sure to convert the first letter to capital case, such as "Grass Skiing"
before you run the Cypher query.

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

In [ ]:
cypher_chain.invoke({"query": "What is a standard technique in alpine skiing?"})

In [ ]:
cypher_chain.invoke({"query": "Tell me about ski touring."})

In [ ]:
cypher_chain.invoke({"query": "Tell me about the biathalon."})

In [ ]:
cypher_chain.invoke({"query": "Tell me about any ski championships."})